In [2]:
import os
import yaml
from yaml import SafeLoader

from qiskit.aqua.algorithms import NumPyEigensolver
import numpy as np
from qiskit.chemistry import FermionicOperator


In [6]:
def get_spatial_integrals(one_electron,two_electron,n_orb):
    one_electron_spatial_integrals = np.zeros((n_orb, n_orb))
    two_electron_spatial_integrals = np.zeros((n_orb, n_orb, n_orb, n_orb))

    for ind, val in enumerate(one_electron):
        # This is because python index starts at 0
        i = int(val[0] - 1)
        j = int(val[1] - 1)
        one_electron_spatial_integrals[i, j] = val[2]
        if i != j:
            one_electron_spatial_integrals[j, i] = val[2]

    for ind, val in enumerate(two_electron):
        i = int(val[0]-1)
        j = int(val[1]-1)
        k = int(val[2]-1)
        l = int(val[3]-1)
        two_electron_spatial_integrals[i, j, k, l] = val[4]
        if two_electron_spatial_integrals[k, l, i, j] == 0:
            two_electron_spatial_integrals[k, l, i, j] = val[4]
        if two_electron_spatial_integrals[i, j, l, k] == 0:
            two_electron_spatial_integrals[i, j, l, k] = val[4]
        if two_electron_spatial_integrals[l, k, i, j] == 0:
            two_electron_spatial_integrals[l, k, i, j] = val[4]
        if two_electron_spatial_integrals[j, i, k, l] == 0:
            two_electron_spatial_integrals[j, i, k, l] = val[4]
        if two_electron_spatial_integrals[k, l, j, i] == 0:
            two_electron_spatial_integrals[k, l, j, i] = val[4]
        if two_electron_spatial_integrals[j, i, l, k] == 0:
            two_electron_spatial_integrals[j, i, l, k] = val[4]
        if two_electron_spatial_integrals[l, k, j, i] == 0:
            two_electron_spatial_integrals[l, k, j, i] = val[4]

    return one_electron_spatial_integrals, two_electron_spatial_integrals

def convert_to_spin_index(one_electron, two_electron,n_orb):
    h1 = np.block([[one_electron, np.zeros((int(n_orb), int(n_orb)))],
                   [np.zeros((int(n_orb), int(n_orb))), one_electron]])
    h2 = np.zeros((2 * n_orb, 2 * n_orb, 2 * n_orb, 2 * n_orb))

    for i in range(len(two_electron)):
        for j in range(len(two_electron)):
            for k in range(len(two_electron)):
                for l in range(len(two_electron)):

                    h2[i,j, k + n_orb, l + n_orb] = two_electron[i, j, k, l]
                    h2[i + n_orb, j + n_orb,k, l] = two_electron[i, j, k, l]

                    if i!=k and j!=l:
                        h2[i,j,k,l] = two_electron[i,j,k,l]
                        h2[i + n_orb, j + n_orb, k + n_orb, l + n_orb] = two_electron[i, j, k, l]
    return h1, 0.5*h2

In [ ]:
data = yaml.load(open("LiH.yaml","r"),SafeLoader)
n_electrons = data['integral_sets'][0]['n_electrons']
n_spatial_orbitals = data['integral_sets'][0]['n_orbitals']
nuclear_repulsion_energy = data['integral_sets'][0]['coulomb_repulsion']['value']

one_electron_import = data['integral_sets'][0]['hamiltonian']['one_electron_integrals']['values']
two_electron_import = data['integral_sets'][0]['hamiltonian']['two_electron_integrals']['values']

one_electron_spatial_integrals, two_electron_spatial_integrals = get_spatial_integrals(one_electron_import,two_electron_import,n_spatial_orbitals)

h1, h2 = convert_to_spin_index(one_electron_spatial_integrals,two_electron_spatial_integrals,n_spatial_orbitals)

# Mapping can be “jordan_wigner”, “parity”, “bravyi_kitaev”, “bksf”
qubitOp = FermionicOperator(h1, h2).mapping(map_type='jordan_wigner')

Print the lowest eigenvalue. To get the eigenvalues of the excited states, more eigen values need to be requested by the NumPyEigensolver.

In [13]:
exact_solution = NumPyEigensolver(qubitOp).run()
print("Exact Result:", np.real(exact_solution.eigenvalues) + nuclear_repulsion_energy)

Exact Result: [-7.88232438]
